In [1]:
import pandas as pd
import os, sys
import nltk
import re
# import seaborn as sns
sys.path.append(".")
import preprocessing.cleaning_functions
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# from wordcloud import WordCloud

import matplotlib.gridspec as gridspec
from matplotlib.ticker import MaxNLocator
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt

import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from plotly.offline import plot

import torch


cwd = os.getcwd()
data_path = os.path.join(cwd, "data")

C:\Users\mathi\.conda\envs\nltk_torch\lib\site-packages\tqdm\auto.py:22: TqdmWarning:

IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html



## Tripadvisor data

In [16]:
tripadvisor_df = pd.read_csv(os.path.join(data_path,"tripadvisor_hotel_reviews.csv"))
tripadvisor_df["Rating"].value_counts()

5    9054
4    6039
3    2184
2    1793
1    1421
Name: Rating, dtype: int64

We can see that the dataset is unbalanced, with much higher numbers for the higher grades. To make this a binary classification problem, i choose to count the numbers above 3 as positive (1) and the numbers 3 and below negative (0).

In [17]:
tripadvisor_df["sentiment"] = pd.cut(x = tripadvisor_df["Rating"], bins = [0, 3, 6], labels = [0, 1])
value_counts = tripadvisor_df["sentiment"].value_counts()
tripadvisor_df = tripadvisor_df.rename(columns={"Review":"review", "Rating":"rating"})

In [18]:
# fig = px.histogram(tripadvisor_df, x="sentiment", width=500, height=250)
# fig.update_layout(bargap=0.3)
# fig.show()

This plot illustrates that the data is unbalanced. By looking into some of the reviews it looks like the reviews are not complete sentences.

In [19]:
X = tripadvisor_df["review"].copy()
y = tripadvisor_df["sentiment"].copy()

In [20]:
X_clean = X.apply(preprocessing.cleaning_functions.clean_data)
X_clean.head()

0    nice hotel expensive parking got good deal sta...
1    ok nothing special charge diamond member hilto...
2    nice rooms * experience hotel monaco seattle g...
3    unique, great stay, wonderful time hotel monac...
4    great stay great stay, went seahawk game aweso...
Name: review, dtype: object

In [ ]:
# from transf